In [4]:
import warnings
warnings.filterwarnings("ignore")

import os, pandas as pd, matplotlib.pyplot as plt, numpy as np, seaborn as sns, glob
from pynwb import NWBHDF5IO
from pathlib import Path
from convnwb.io import load_nwbfile


helper functions

In [14]:
# Function to handle spike filtering, aligning, truncating, and FR calculation
def process_spikes(neuron_row, trial_df):

    trials_spikes_filtered, trials_spikes_adjusted = [], []
    trials_num_spikes, trials_RT, trials_FR = [], [], []
    trials_smooth_spikes, trials_smooth_FR = [], []

    for _, trial in trial_df.iterrows():
        
        # 1. Filter spike times within trial's start and stop times
        trial_spikes = neuron_row['spike_times'][
            (neuron_row['spike_times'] >= trial['start_time']) &
            (neuron_row['spike_times'] <= trial['stop_time'])
        ]
        
        # 2. align spike times to start & truncated to 5s
        trial_spikes_aligned = trial_spikes - trial['start_time']

        # 3. num_spikes
        trial_num_spikes = len(trial_spikes_aligned)

        # 4. reaction time
        trial_RT = trial['stop_time'] - trial['start_time']
        
        # 5. firing rate
        trial_FR = len(trial_spikes_aligned)/trial_RT

        # 6. smooth spikes

        window_size, step = 1, .5

        trial_smooth_spikes = []
        for i in np.arange(0, 5-window_size+step, step):
            trial_smooth_spikes.append(len(trial_spikes_aligned[(trial_spikes_aligned >= i) & (trial_spikes_aligned < i + window_size)]))
        trial_smooth_spikes = np.array(trial_smooth_spikes)

        # 7. FR of smooth spikes
        trial_smooth_FR = np.mean(trial_smooth_spikes)
        
        # Append results to lists
        trials_spikes_filtered.append(trial_spikes)
        trials_spikes_adjusted.append(trial_spikes_aligned)
        trials_num_spikes.append(trial_num_spikes)
        trials_RT.append(trial_RT)
        trials_FR.append(trial_FR)
        trials_smooth_spikes.append(trial_smooth_spikes)
        trials_smooth_FR.append(trial_smooth_FR)

    return trials_spikes_filtered, trials_spikes_adjusted, trials_num_spikes, trials_RT, trials_FR, trials_smooth_spikes, trials_smooth_FR


load example data file

In [15]:
sess_file = sorted(glob.glob('../data/000623/sub-*/*.nwb'))[2]
nwbfile, io = load_nwbfile(os.path.basename(sess_file), os.path.dirname(sess_file), return_io=True)
nwbfile

,location,group,group_name,origchannel,pairwise_distances,origchannel_name,x,y,z
id,,,,,,,,,
0,Left ACC,Neuralynx-Atlas-microwire-1 pynwb.ecephys.ElectrodeGroup at 0x139662936195216\nFields:\n description: Behnke Fried/Micro Inner Wire Bundle (Behnke-Fried BF08R-SP05X-000 and WB09R-SP00X-0B6; Ad-Tech Medical)\n device: Neuralynx-Atlas pynwb.device.Device at 0x139662900000272\nFields:\n description: CS - Neuralynx-Atlas\n\n location: Left ACC\n,Neuralynx-Atlas-microwire-1,micro-1,NA,LACC1,-0.76,28.11,20.35
1,Left ACC,Neuralynx-Atlas-microwire-2 pynwb.ecephys.ElectrodeGroup at 0x139662936195920\nFields:\n description: Behnke Fried/Micro Inner Wire Bundle (Behnke-Fried BF08R-SP05X-000 and WB09R-SP00X-0B6; Ad-Tech Medical)\n device: Neuralynx-Atlas pynwb.device.Device at 0x139662900000272\nFields:\n description: CS - Neuralynx-Atlas\n\n location: Left ACC\n,Neuralynx-Atlas-microwire-2,micro-2,NA,LACC2,-0.76,28.11,20.35
2,Left ACC,Neuralynx-Atlas-microwire-3 pynwb.ecephys.ElectrodeGroup at 0x139662936209424\nFields:\n description: Behnke Fried/Micro Inner Wire Bundle (Behnke-Fried BF08R-SP05X-000 and WB09R-SP00X-0B6; Ad-Tech Medical)\n device: Neuralynx-Atlas pynwb.device.Device at 0x139662900000272\nFields:\n description: CS - Neuralynx-Atlas\n\n location: Left ACC\n,Neuralynx-Atlas-microwire-3,micro-3,NA,LACC3,-0.76,28.11,20.35
3,Left ACC,Neuralynx-Atlas-microwire-4 pynwb.ecephys.ElectrodeGroup at 0x139662936210576\nFields:\n description: Behnke Fried/Micro Inner Wire Bundle (Behnke-Fried BF08R-SP05X-000 and WB09R-SP00X-0B6; Ad-Tech Medical)\n device: Neuralynx-Atlas pynwb.device.Device at 0x139662900000272\nFields:\n description: CS - Neuralynx-Atlas\n\n location: Left ACC\n,Neuralynx-Atlas-microwire-4,micro-4,NA,LACC4,-0.76,28.11,20.35
,location,group,group_name,origchannel,pairwise_distances,origchannel_name,x,y,z
id,,,,,,,,,
0,Left ACC,Neuralynx-Atlas-microwire-1 pynwb.ecephys.ElectrodeGroup at 0x139662936195216\nFields:\n description: Behnke Fried/Micro Inner Wire Bundle (Behnke-Fried BF08R-SP05X-000 and WB09R-SP00X-0B6; Ad-Tech Medical)\n device: Neuralynx-Atlas pynwb.device.Device at 0x139662900000272\nFields:\n description: CS - Neuralynx-Atlas\n\n location: Left ACC\n,Neuralynx-Atlas-microwire-1,micro-1,NA,LACC1,-0.76,28.11,20.35
1,Left ACC,Neuralynx-Atlas-microwire-2 pynwb.ecephys.ElectrodeGroup at 0x139662936195920\nFields:\n description: Behnke Fried/Micro Inner Wire Bundle (Behnke-Fried BF08R-SP05X-000 and WB09R-SP00X-0B6; Ad-Tech Medical)\n device: Neuralynx-Atlas pynwb.device.Device at 0x139662900000272\nFields:\n description: CS - Neuralynx-Atlas\n\n location: Left ACC\n,Neuralynx-Atlas-microwire-2,micro-2,NA,LACC2,-0.76,28.11,20.35
2,Left ACC,Neuralynx-Atlas-microwire-3 pynwb.ecephys.ElectrodeGroup at 0x139662936209424\nFields:\n description: Behnke Fried/Micro Inner Wire Bundle (Behnke-Fried BF08R-SP05X-000 and WB09R-SP00X-0B6; Ad-Tech Medical)\n device: Neuralynx-Atlas pynwb.device.Device at 0x139662900000272\nFields:\n description: CS - Neuralynx-Atlas\n\n location: Left ACC\n,Neuralynx-Atlas-microwire-3,micro-3,NA,LACC3,-0.76,28.11,20.35


consolidate dataframe

In [16]:
# skip subj1
files = sorted(glob.glob('../data/000623/sub-*/*.nwb'))
all_neur_tri_df = []
count = 0

# create table comprising n_rows = n_neurs * n_trials = 1450 * 41
for file in files:
    
    nwbfile, io = load_nwbfile(os.path.basename(file), os.path.dirname(file), return_io=True)
    subj = os.path.basename(file)[:8]

    # trial data per session
    trial_df = nwbfile.trials.to_dataframe()

    # all neurons within session
    neur_df = nwbfile.units.to_dataframe()[['unit_id_session', 'electrode_id', 'spike_times']]
    
    neur_df['subj'] = neur_df['unit_id_session'].str[:3]
    neur_df['sess'] = neur_df['unit_id_session'].str[:8]
    neur_df = neur_df.rename(columns={'unit_id_session': 'neuron'})

    # electrode info
    electrodes_df = nwbfile.electrodes.to_dataframe()
    
    # Replace unit_id in neur_df with the corresponding location from electrodes_df
    neur_df['electrode_id'] = neur_df['electrode_id'].map(electrodes_df['location'])
    neur_df['electrode_id'] = neur_df['electrode_id'].str.split().str[-1]
    neur_df = neur_df.rename(columns={'electrode_id': 'region'})

    print(f'{subj} neurons {len(neur_df)}')

    for neuron in range(len(neur_df)):
    
        # get specific neuron's spike info
        neuron_row = neur_df.iloc[neuron]

        # Repeat the neuron row as many times as there are trials
        repeated_neuron_row = pd.DataFrame([neuron_row] * len(trial_df)).reset_index(drop=True)

        # Concatenate the neuron data with the trial data horizontally
        neur_tri_df = pd.concat([repeated_neuron_row, trial_df.reset_index(drop=True)], axis=1)
        
        (
            neur_tri_df['trial_spikes'],  # within trial spikes
            neur_tri_df['trial_aligned_spikes'],  # spikes aligned to start
            neur_tri_df['num_spikes'],  # number of spikes
            neur_tri_df['RT'],  # reaction time
            neur_tri_df['FR'],  # len(spikes)/RT
            neur_tri_df['trial_smooth_spikes'],  # spikes smoothed using sliding window
            neur_tri_df['trial_smooth_FR']  # FR of smoothed spikes
        ) = process_spikes(neuron_row, trial_df)


        # Finally, append the neur_tri_df for this neuron to the overall df
        all_neur_tri_df.append(neur_tri_df)

    
all_neur_tri_df = pd.concat(all_neur_tri_df, ignore_index=True)

# reorder cols
all_neur_tri_df = all_neur_tri_df[['subj', 'sess', 'neuron', 'region',
                                   'start_time', 'stop_time', 'RT',
                                   'trial_spikes', 'trial_aligned_spikes', 'num_spikes', 'FR',
                                   'trial_smooth_spikes', 'trial_smooth_FR',
                                   'stim_phase', 'stimulus_file',
                                   'response_correct', 'response_confidence', 'actual_response']]

all_neur_tri_df.to_pickle('all_neur_tri_df.pkl')
    

sub-CS41 neurons 7
sub-CS41 neurons 7
sub-CS42 neurons 50
sub-CS42 neurons 46
sub-CS43 neurons 32
sub-CS43 neurons 32
sub-CS44 neurons 49
sub-CS47 neurons 75
sub-CS47 neurons 73
sub-CS48 neurons 143
sub-CS48 neurons 114
sub-CS49 neurons 12
sub-CS49 neurons 14
sub-CS51 neurons 30
sub-CS51 neurons 46
sub-CS53 neurons 34
sub-CS53 neurons 48
sub-CS54 neurons 123
sub-CS54 neurons 112
sub-CS55 neurons 57
sub-CS55 neurons 29
sub-CS56 neurons 30
sub-CS56 neurons 29
sub-CS57 neurons 33
sub-CS57 neurons 28
sub-CS58 neurons 89
sub-CS60 neurons 27
sub-CS62 neurons 44
sub-CS62 neurons 44
